In [1]:
# importing depedencies
import pandas as pd
import geopandas as gpd
import folium
import json
import vincent
from altair import Chart
from pprint import pprint
from pymongo import MongoClient
from datetime import datetime
from folium.plugins import HeatMap

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
db = mongo['forestFireProject_db']

### import data

In [38]:
#set some data variables for filtering
myMonth = 8
myYear = 2018
WaterfromDate = datetime(myYear-1, myMonth + 1, 1)
WaterfromDate=WaterfromDate.strftime('%Y-%m-%d')
WatertoDate = datetime(myYear+1, myMonth,1)
WatertoDate=WatertoDate.strftime('%Y-%m-%d %H:%M')

FirefromDate = datetime(myYear, myMonth, 1)
FirefromDate= FirefromDate.strftime('%Y-%m-%d')
# fromDate = pd.Timestamp(fromDate)
FiretoDate = datetime(myYear, myMonth+1,1)
FiretoDate=FiretoDate.strftime('%Y-%m-%d %H:%M')


In [39]:
# import shape file defining Canadian water regions
WaterRegions_gdf = gpd.read_file('resources/Drainage_regions_reduced.shp')



In [12]:
WaterRegions_gdf=WaterRegions_gdf[['DR_Code', 'DR_Name', 'ODA_Code', 'ODA_Name', 'geometry']]


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DR_Code   25 non-null     object  
 1   DR_Name   25 non-null     object  
 2   ODA_Code  25 non-null     object  
 3   ODA_Name  25 non-null     object  
 4   geometry  25 non-null     geometry
dtypes: geometry(1), object(4)
memory usage: 1.1+ KB


In [13]:
#create some user friendly column names for presentation
WaterRegions_gdf.rename(columns = {'DR_Name':'Drainage Regions', 'ODA_Name':'Ocean Drainage Areas'}, inplace = True)


In [14]:
#step through the mongodb collections, convert the mongodb _id to str so it can be serialized by Geopandas and store the collection in a Dictionary 
#dataframes for further processing.
myCollectionsList = db.list_collection_names()
myDataFrameDict = {}
query = {}
for collections in myCollectionsList:
    thisCollection = db[collections]
    if collections == 'ForestFirePoints':
        query = {"REP_DATE":{'$gte':FirefromDate,
                             '$lte':FiretoDate}}
    
        # query = {'$and':[{"YEAR":fireMonth},{"MONTH":fireYear}]}
    elif collections == 'WaterSampleData':
        query = {"DATE_TIME_HEURE":{'$gte':WaterfromDate,
                                    '$lte':WatertoDate}}
    
    myDocList = list(thisCollection.find(query))
    for document in myDocList:
        document['_id'] = str(document['_id'])
    thisDataFrame = pd.DataFrame(myDocList)
    query = {}
    myDataFrameDict[collections + '_df']=thisDataFrame

In [20]:
#Slicing out superflous columns.
myDataFrameDict['WaterSampleData_df']=myDataFrameDict['WaterSampleData_df'].iloc[:,:10]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301420 entries, 0 to 301419
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   _id              301420 non-null  object
 1   SITE_NO          301420 non-null  object
 2   DATE_TIME_HEURE  301420 non-null  object
 3   FLAG_MARQUEUR    301420 non-null  object
 4   VALUE_VALEUR     301420 non-null  object
 5   SDL_LDE          301420 non-null  object
 6   MDL_LDM          301420 non-null  object
 7   VMV_CODE         301420 non-null  int64 
 8   UNIT_UNITÉ       301025 non-null  object
 9   VARIABLE         301420 non-null  object
dtypes: int64(1), object(9)
memory usage: 23.0+ MB


In [21]:
#merge SampleData and the Watersites for plotting
WaterSampleData_df = myDataFrameDict['WaterSampleData_df']
WaterCombinedData_df =  pd.merge(WaterSampleData_df, myDataFrameDict['WaterSites_df'], how='inner', on='SITE_NO')


In [22]:
#define geometry columns 
WaterCombinedData_df['geometry'] = gpd.points_from_xy(WaterCombinedData_df['LONGITUDE'],WaterCombinedData_df['LATITUDE'])
myDataFrameDict["ForestFirePoints_df"]['geometry'] = gpd.points_from_xy(myDataFrameDict["ForestFirePoints_df"]['LONGITUDE'], myDataFrameDict["ForestFirePoints_df"]['LATITUDE'])

In [23]:
myDataFrameDict["ForestFirePoints_df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   _id         124 non-null    object  
 1   FID         124 non-null    int64   
 2   SRC_AGENCY  124 non-null    object  
 3   FIRE_ID     124 non-null    object  
 4   FIRENAME    124 non-null    object  
 5   LATITUDE    124 non-null    float64 
 6   LONGITUDE   124 non-null    float64 
 7   YEAR        124 non-null    int64   
 8   MONTH       124 non-null    int64   
 9   DAY         124 non-null    int64   
 10  REP_DATE    124 non-null    object  
 11  ATTK_DATE   124 non-null    object  
 12  OUT_DATE    124 non-null    object  
 13  DECADE      124 non-null    object  
 14  SIZE_HA     124 non-null    float64 
 15  CAUSE       124 non-null    object  
 16  PROTZONE    124 non-null    object  
 17  FIRE_TYPE   124 non-null    object  
 18  MORE_INFO   124 non-null    object  
 19  CFS_REF_

In [24]:
#convert the df's to a gdf's
FirePoint_gdf =  gpd.GeoDataFrame(myDataFrameDict["ForestFirePoints_df"],geometry='geometry')
FirePoint_gdf.set_crs(epsg=4617, inplace=True)
WaterCombinedData_gdf = gpd.GeoDataFrame(WaterCombinedData_df, geometry='geometry')
WaterCombinedData_gdf.set_crs(epsg=4617, inplace=True)


,_id_x,SITE_NO,DATE_TIME_HEURE,FLAG_MARQUEUR,VALUE_VALEUR,SDL_LDE,MDL_LDM,VMV_CODE,UNIT_UNITÉ,VARIABLE,...,PROV_TERR,PROV_TERR_FR,PEARSEDA,PEARSEDA_FR,OCEANDA,OCEANDA_FR,DATA_URL,DATA_URL_FR,OPEN_DATA_URL,geometry
0,65e3b4afdafd628390ca347d,NW06MA0001,2017-09-13 14:45,,10.5,0.5,,107941,µG/L,ALUMINUM DISSOLVED,...,NVT.,NT,KEEWATIN-SOUTHERN BAFFIN ISLAND,KEEWATIN-SUD DE L'ÎLE DE BAFFIN,HUDSON BAY,BAIE D'HUDSON,,,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-97.06780 64.77310)
1,65e3b4afdafd628390ca347f,NW06MA0001,2017-09-13 14:45,,18.1,0.05,,107944,µG/L,BARIUM DISSOLVED,...,NVT.,NT,KEEWATIN-SOUTHERN BAFFIN ISLAND,KEEWATIN-SUD DE L'ÎLE DE BAFFIN,HUDSON BAY,BAIE D'HUDSON,,,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-97.06780 64.77310)
2,65e3b4afdafd628390ca3480,NW06MA0001,2017-09-13 14:45,,19.1,0.05,,107908,µG/L,BARIUM TOTAL,...,NVT.,NT,KEEWATIN-SOUTHERN BAFFIN ISLAND,KEEWATIN-SUD DE L'ÎLE DE BAFFIN,HUDSON BAY,BAIE D'HUDSON,,,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-97.06780 64.77310)
3,65e3b4afdafd628390ca3481,NW06MA0001,2017-09-13 14:45,,7.8,0.5,,10101,MG/L,ALKALINITY TOTAL CACO3,...,NVT.,NT,KEEWATIN-SOUTHERN BAFFIN ISLAND,KEEWATIN-SUD DE L'ÎLE DE BAFFIN,HUDSON BAY,BAIE D'HUDSON,,,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-97.06780 64.77310)
4,65e3b4afdafd628390ca3482,NW06MA0001,2017-09-13 14:45,,0.004,0.001,,107945,µG/L,BERYLLIUM DISSOLVED,...,NVT.,NT,KEEWATIN-SOUTHERN BAFFIN ISLAND,KEEWATIN-SUD DE L'ÎLE DE BAFFIN,HUDSON BAY,BAIE D'HUDSON,,,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-97.06780 64.77310)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301415,65e3b7306116da18f805df6c,YT09FC0002,2019-07-22 11:30,,22.9,0.05,,2074,NTU,TURBIDITY,...,Y.T.,YN,YUKON,YUKON,PACIFIC OCEAN,OCÉAN PACIFIQUE,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-139.80127 67.58065)
301416,65e3b7306116da18f805df6d,YT09FC0002,2019-07-22 11:30,,0.56,0.0005,,107935,µG/L,URANIUM TOTAL,...,Y.T.,YN,YUKON,YUKON,PACIFIC OCEAN,OCÉAN PACIFIQUE,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-139.80127 67.58065)
301417,65e3b7306116da18f805df6e,YT09FC0002,2019-07-22 11:30,,1.54,0.01,,107936,µG/L,VANADIUM TOTAL,...,Y.T.,YN,YUKON,YUKON,PACIFIC OCEAN,OCÉAN PACIFIQUE,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-139.80127 67.58065)
301418,65e3b7306116da18f805df6f,YT09FC0002,2019-07-22 11:30,,4.0,0.2,,107939,µG/L,ZINC TOTAL,...,Y.T.,YN,YUKON,YUKON,PACIFIC OCEAN,OCÉAN PACIFIQUE,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTP://AQUATIC.PYR.EC.GC.CA/WEBDATAONLINENATIO...,HTTPS://DATA-DONNEES.EC.GC.CA/DATA/SUBSTANCES/...,POINT (-139.80127 67.58065)


In [25]:
#lets reduce the WaterSites we're plotting to the west
fromSites = WaterCombinedData_gdf['SITE_NO'].unique()
popSites = []
for site in fromSites:
    # if site[:2] in ['BC', 'AL', 'SA']:
    popSites.append(site)


In [26]:
#get our popup data for our choosen sites

staging_gdf = WaterCombinedData_gdf.loc[(WaterCombinedData_gdf['SITE_NO'].isin(popSites) & WaterCombinedData_gdf['VARIABLE'].isin(['ARSENIC TOTAL','CADMIUM TOTAL', 'LEAD TOTAL']))]
# staging_gdf = WaterCombinedData_gdf.loc[(WaterCombinedData_gdf['VARIABLE'].isin(['ARSENIC TOTAL','CADMIUM TOTAL', 'LEAD TOTAL']))]

# create a list of sites with a list of VARIABLES we want to plot
PlotWaterData_gdf = staging_gdf[['SITE_NO','DATE_TIME_HEURE','VALUE_VALEUR', 'VARIABLE', 'geometry']].copy()

sitePlotData_gdf = staging_gdf[['SITE_NO','SITE_NAME','LATITUDE','LONGITUDE', 'geometry']].copy()
sitePlotData_gdf.drop_duplicates(inplace = True)

In [27]:
# find the limits of the data
maxlat = FirePoint_gdf['LATITUDE'].max()
minlat = FirePoint_gdf['LATITUDE'].min()
maxlng = FirePoint_gdf['LONGITUDE'].max()
minlng = FirePoint_gdf['LONGITUDE'].min()


In [35]:
myMap = WaterRegions_gdf[['Ocean Drainage Areas', 'Drainage Regions', 'geometry']].explore(column = 'Ocean Drainage Areas',
                                 name = "Drainage Areas", 
                                 overlay = False, 
                                 cmap = 'Accent', 
                                 zoom_start = 4,
                                 zoom_on_click=False,
                                 min_lat=minlat,
                                 max_lat=maxlat,
                                 min_lon=minlng,
                                 max_lon=maxlng,
                                 tooltip = False,
                                 attr = "https://open.canada.ca/en/open-government-licence-canada",
                                 popup = ['Ocean Drainage Areas', 'Drainage Regions'])


#create a heat map based on the size of the fire
HeatMap(FirePoint_gdf[['LATITUDE', 'LONGITUDE', 'SIZE_HA']], name='Forest Fire Density', overlay = True, show=False).add_to(myMap)

#create a feature layer for the forest fire daa.
folium.GeoJson(FirePoint_gdf,
               name = "Forest Fires", 
               marker = folium.Marker(icon=folium.Icon(icon='fire', color='red')),
               popup=folium.GeoJsonPopup(fields = ['FID', 'FIRENAME', 'SIZE_HA', 'MORE_INFO']),
               show = False,
               overlay = True,    
              ).add_to(myMap)


#Create a Feature Group for each water monitoring site.
WaterSiteFG = folium.FeatureGroup(name="Water Data Sites", show=False).add_to(myMap)
for i, row in sitePlotData_gdf.iterrows():
    #create the chart for the values we are plotting for each site.
    line = (
        Chart(PlotWaterData_gdf.loc[PlotWaterData_gdf['SITE_NO'] == row['SITE_NO']])
        .mark_line()
        .encode(
            x="DATE_TIME_HEURE",
            y="VALUE_VALEUR",
            color="VARIABLE",
            # size="VALUE_VALEUR"
        )
    )
    title= ("Heavy Metal Concentrations")
    
    vega_lite=folium.VegaLite(
        line,
        width="100%",
        height="100%",
   
       
    )
    #add the chart to a Marker and add that marker to a feature group for plotting.
    mylat = row['LATITUDE']
    mylng = row['LONGITUDE']

    loc = [mylat, mylng]
    myMarker = folium.Marker(loc, icon=folium.Icon(icon='cloud', color='green'))
    myToolTip = folium.Tooltip(f"Site: {row['SITE_NO']},\\n Site Name:{row['SITE_NAME']}") 
    myPopUp = folium.Popup()
    vega_lite.add_to(myPopUp)
    myPopUp.add_to(myMarker)
    myToolTip.add_to(myMarker)
    myMarker.add_to(WaterSiteFG)
    


folium.TileLayer("OpenStreetMap").add_to(myMap)
folium.LayerControl().add_to(myMap)

In [ ]:
myMap